In [ ]:
import re
import numpy as np
import pandas as pd
import spacy
from pprint import pprint
import math
import io
import gensim
import csv
import warnings

#suppress warnings
warnings.filterwarnings('ignore')

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install keybert
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_lg

In [ ]:
def cleaning_dataset(JD):
# Convert to list
  data = JD.values.tolist()
  # Remove new line characters
  data = [re.sub('\s+', ' ', sent) for sent in data]
  # Remove Emails
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
  # Remove distracting single quotes
  data = [re.sub("\'", "", sent) for sent in data]
  return data

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
      # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
# Convert a document in a list of token

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ 
        not in ['-PRON-'] else '' for token in 
        doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
!python -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb

2022-08-11 13:04:39.056064: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(data_lemmatized):
  vec = TfidfVectorizer(stop_words="english")
  X = vec.fit_transform(data_lemmatized)
  features = vec.get_feature_names_out()
  return [X,features]
  

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

def topic_modeling(vector):
  lda = LatentDirichletAllocation(n_components = 25, random_state=None, 
                                 doc_topic_prior = 1.25,topic_word_prior = 0.09)
  # doc_topic_prior = alpha
  # topic_word_prior = beta
  theta = lda.fit_transform(vector)
  return [theta,lda.components_,lda]

In [ ]:
def document_per_topic(lda_component,features):
  for tid, topic in enumerate(lda_component):
    print('Topic ID: ', tid)
    print('Word IDs: ', topic.argsort()[:-15:-1])
    print('Word prob.: ', [topic[i] for i in topic.argsort()[:-15:-1]])
    print('Words: ', [features  [i] for i in topic.argsort()[:-15:-1]])
  return True

In [ ]:
def topic_per_word(theta,lda_component,features):
  for doc_id, tid in enumerate(theta):
      print("JD ID: ", doc_id)
      print('Topic IDs: ', tid.argsort()[-10:-1])
      for i in tid.argsort()[-10:-1]:
        print("The words in the topic ",i, "are: ",[features[j] for j in lda_component[i].argsort()[-20:-1]])
  return True

In [ ]:
def top_20_words_per_document(theta,lda_components,features):
  dic_doc_words = {}
  for doc_id, tid in enumerate(theta):
      dic_doc_words[doc_id] = []
      for i in tid.argsort()[-5:-1]:
        n = 1
        l = []
        for j in lda_components[i].argsort()[-20:-1]:
            if n < 20:
              l.append(features[j])
              n += 1
        dic_doc_words[doc_id].append(l)
  new_dic_doc = {}
  for key,values in dic_doc_words.items():
    text = ""
    for value in values:
      for v in value:
        text += " "
        text += v
    new_dic_doc[key] = text
  
  return new_dic_doc

In [ ]:
def evaluation(lda,vector):
  # Log Likelyhood: Higher the better
  print("Log Likelihood: ", lda.score(vector))
  # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
  print("Perplexity: ", lda.perplexity(vector))
  return True

In [ ]:
def list_of_words(theta,lda_components):
  dic_doc_words = {}
  for doc_id, tid in enumerate(theta):
      dic_doc_words[doc_id] = []
      for i in tid.argsort()[-5:-1]:
        n = 1
        l = []
        for j in lda_components[i].argsort()[-20:-1]:
            if n < 20:
              l.append(features[j])
              n += 1
        dic_doc_words[doc_id].append(l)
  word_list = []
  for key,value in dic_doc_words.items():
    for v in value:
      for v1 in v:
        word_list.append(v1)
  return word_list

In [ ]:
def JD_ID(jd):
  dic_jd = {}
  count = 0
  for i in jd:
    dic_jd[count] = i
    count += 1
  return dic_jd

In [ ]:
def merge_documents(corpus):
  documents = []
  for key,value in corpus.items():
    l = value.split()
    for i in l:
      documents.append(i.lower())
  return documents

In [ ]:
def topic_coherence(word_list,documents):
  
  ans = 0
  count = 0
  for word in word_list:
    
    wi = 0
    wi_wj = 0
    if len(word_list) != count+1:
      next_word = word_list[count+1]
      # print(next_word)
      flag = False
      for doc in documents:
        if word == doc:
          wi += 1  
          flag = True
        
        if next_word == doc and flag == True:
          flag = False
          wi_wj += 1
      
        # print("iteration")
      count += 1
      if wi == 0:
          wi =1
      sum = (wi_wj+1) / wi
      ans += math.log(sum)
      
    else:
      continue

  return ans

In [ ]:
def courses_dic(JD_title,JD_outlines):
  JD_dic = {}
  for i in range(len(JD_title)):
    JD_dic[JD_title[i]] = JD_outlines[i]

  new_JD_dic = {}
  for title, outline in JD_dic.items():
    if len(str(outline)) < 5:
      pass
    else:
      new_JD_dic[title] = outline
  return new_JD_dic

In [ ]:
def Jaccard_Similarity(doc1, doc2): 
    
    # List the unique words in a document
    words_doc1 = set(doc1.split()) 
    words_doc2 = set(doc2.split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [ ]:
def maintain_outline_list(list_out,outline,score):
  if len(list_out) < 3:
    return list_out.append([outline,score])

  else:
    for i in list_out:
      if i[1] < score:
        list_out.remove(i)
        list_out.append([outline,score])
        return list_out

In [ ]:
from itertools import islice
def topic_for_course(dic_new_outline,list_out,title,job_desc,score):
  if len(list_out) < 4:
    dic_new_outline[title] = job_desc
    return dic_new_outline

  else:
    for i in list_out:
      count = 0
      if i[1] < score:
        del dic_new_outline[next(islice(dic_new_outline, count, None))]
        dic_new_outline[title] = job_desc
        count += 1
      else:
        count+= 1    
    return dic_new_outline
      

In [ ]:
def words_from_jaccard_similarity(top_words_per_document,JD_dic):
  jaccard_similarity= {}
  course_title_new_topics = {}
  for key,job_desc in top_words_per_document.items():
    max_score = False
    course_titles = []
    dic_outline = {}
    for title,outlines in JD_dic.items():
      score = Jaccard_Similarity(job_desc,outlines)
      if (max_score < score):
        max_score = score
        maintain_outline_list(course_titles,title,max_score)
        topic_for_course(dic_outline,course_titles,title,job_desc,max_score)
    jaccard_similarity[key] = course_titles 
    course_title_new_topics[key] = dic_outline
  return [jaccard_similarity,course_title_new_topics]

In [ ]:
def Cosine_Similarity(a, b):
   return dot(a, b)/(norm(a)*norm(b))

In [ ]:
# def create_cvs_file_of_Similarity(name,top_words_per_document,similarity)
# with open('Outline_new_content.csv','w',encoding='UTF8',newline="") as f:
#     header = ['Title','New Content']
#     writer = csv.writer(f)
#     writer.writerow(header)
#     new_content = course_outline_new_content(top_words_per_document,similarity)
#     for key,value in new_content.items():
#         string = ""
#         for v in value:
#             string += v+', '
#         writer.writerow([key,string])

In [ ]:
def words_from_cosine_similarity(top_words_per_document,JD_dic):
  cosine_similarity= {}
  course_title_new_topics = {}
  for key,job_desc in top_words_per_document.items():
    max_score = False
    course_titles = []
    dic_outline = {}
    nlp_job_desc_vec = nlp(job_desc).vector
    for title,outlines in new_JD_dic.items():

      nlp_outline_vec = nlp(outlines).vector
        
      score = Cosine_Similarity(nlp_job_desc_vec, nlp_outline_vec)
      if (max_score < score):
        max_score = score
        maintain_outline_list(course_titles,title,max_score)
        topic_for_course(dic_outline,course_titles,title,job_desc,max_score)
    cosine_similarity[key] = course_titles 
    course_title_new_topics[key] = dic_outline
  return [cosine_similarity,course_title_new_topics]

In [ ]:
def most_occuring_course(similarity_dic):
  similar_dic = {}
  for key,value in similarity_dic.items():
    for v in value:
      if v[0] not in similar_dic:
        similar_dic[v[0]]=0
      else:
        similar_dic[v[0]]+=1
  return similar_dic

In [ ]:
from collections import Counter

def top_repeated_courses(similar_dic,no_of_courses):

  c = Counter(similar_dic)
  most_common = c.most_common(no_of_courses)  # returns top 3 pairs

  return most_common

In [ ]:
def trending_course_content(repeated_courses,courseTitle_Newtopics):
  with open('Occuring_courses.csv', 'w',encoding = 'UTF8', newline="") as f:
      header = ["Course Name","Jobs no."]
      writer = csv.writer(f)
      writer.writerow(header)
      for i in repeated_courses:
        flag= True 
        for j in i:
          j=list(i)
          course_new_outline = courseTitle_Newtopics[j[0]]
          #print(j)
          l=[j[0],j[1],course_new_outline]
          #l = [i,j]
          if flag == True:
            writer.writerow(l)
            flag = False

In [ ]:
def new_course_content(course_title_new_topics):
  new_content = {}
  for job_id,content_jd in course_title_new_topics.items():
    for course_name, course_new_content in content_jd.items():
      if course_name not in new_content:
        new_content[course_name] = course_new_content
      else:
        new_content[course_name] += course_new_content+" "
        better_content = delete_repetition(new_content[course_name])
        new_content[course_name] = better_content
  return new_content

In [ ]:
def delete_repetition(content):
  unrepeated_content =' '.join(dict.fromkeys(content.split()))
  return unrepeated_content

In [ ]:
#Dataset of Job Descriptions
from google.colab import files

uploaded = files.upload()


MessageError: ignored

In [ ]:
#Dataset of Course Outlines
from google.colab import files


uploaded_CO = files.upload()

In [ ]:
dataset = pd.read_csv(io.BytesIO(uploaded['dataset.csv']), encoding= 'unicode_escape')
JD = dataset["jobdescription"]
print(JD)


data = cleaning_dataset(JD)
data_words = list(sent_to_words(data))
# print(data_words[0])
# break
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

vector_and_features = vectorize(data_lemmatized)
vector = vector_and_features[0]
features = vector_and_features[1]

theta_lda_component_model = topic_modeling(vector)

lda_component = theta_lda_component_model[1]
theta = theta_lda_component_model[0]
lda_model = theta_lda_component_model[2]


# document_per_topic(lda_component,features)

# topic_per_word(theta,lda_component,features)

top_words_per_document = top_20_words_per_document(theta,lda_component,features)

evaluation(lda_model,vector)

jd_id = JD_ID(data)
list_of_documents = merge_documents(jd_id)
word_list = list_of_words(theta,lda_component)
# print(topic_coherence(word_list,list_of_documents))
# print(JD)

In [ ]:
evaluation(lda_model,vector)
print(topic_coherence(word_list,list_of_documents))

In [ ]:
pprint(top_words_per_document)

In [ ]:
course_dataset = pd.read_csv('nu_course_outlines.csv')
print(course_dataset)
JD_outlines = course_dataset['outlines']
JD_title = course_dataset['Title']

JD_dic = courses_dic(JD_title,JD_outlines)
jaccard_similarity = words_from_jaccard_similarity(top_words_per_document,JD_dic)

jaccard_similarity_per_JD = jaccard_similarity[0]
jaccard_similarity_new_topics = jaccard_similarity[1]

trending_courses = top_repeated_courses(most_occuring_course(jaccard_similarity_per_JD),10)

courseTitle_newTopics = new_course_content(jaccard_similarity_new_topics)
# print(courseTitle_newTopics)


In [ ]:
courseTitle_newTopics

In [ ]:
pprint(keyword_extraction_bert(courseTitle_newTopics))

In [ ]:
from keybert import KeyBERT

def keyword_extraction_bert(courseTitle_newTopics):
  keyword_dic = {}
  for title, new_outline in courseTitle_newTopics.items():
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(new_outline,keyphrase_ngram_range=(1, 2),top_n = 20)
    keyword_dic[title] = keywords
  return keyword_dic


In [ ]:
import yake

def keyword_extraction_yake(courseTitle_newTopics):
  keyword_dic = {}
  for title, new_outline in courseTitle_newTopics.items():
  
    kw_extractor = yake.KeywordExtractor(top = 20, n = 2)
    keywords = kw_extractor.extract_keywords(new_outline)
    keyword_dic[title] = []
    for kw in keywords:
      keyword_dic[title].append(kw) 
  return keyword_dic

In [ ]:
pprint(keyword_extraction_yake(courseTitle_newTopics))